In [ ]:
# for visualizing the aformentioned tree
import sys
import matplotlib
matplotlib.use('Agg')

In [ ]:

# for reading and initializing decision tree
import numpy
import pandas

In [ ]:

df = pandas.read_csv("whatever.csv") # reads data set
print(df) # prints the data set

In [ ]:

class tree: 
    def __init__(self):
      self.depth = 0
      self.pEntropy = 0

    def getDepth(self):
      return self.depth
    
    def setDepth(self, depth):
      self.depth = depth

    """
    here is how the tree is created:
    first step: it checks if the labels are the same, if they are, it returns the label
    second step: it checks for the best feature to split the data on
    third step: it splits the data on the best feature
    fourth step: it creates a tree with the split data
    """
    def createTree(self, data, labels):
        if len(numpy.unique(labels)) == 1:
            return {'label': labels[0], 'data': data, 'children': []}

        best_feature = None
        best_igain = 0
        for feature in range(data.shape[1]):
            igain = self.iGain(self.entropy(labels), 
                               [numpy.mean(data[:, feature] == value) for value in numpy.unique(data[:, feature])], 
                               [self.entropy(labels[data[:, feature] == value]) for value in numpy.unique(data[:, feature])])
            if igain > best_igain:
                best_igain = igain
                best_feature = feature

        tree = {'label': None, 'data': data, 'children': []}
        for value in numpy.unique(data[:, best_feature]):
            child_data = data[data[:, best_feature] == value]
            child_labels = labels[data[:, best_feature] == value]
            tree['children'].append({'value': value, 'tree': self.createTree(child_data, child_labels)})

        return tree

    def entropy(self, labels):
      entropy = 0
      unique_labels = numpy.unique(labels) # gets the unique labels
      for label in unique_labels:
          probability = numpy.mean(labels == label) # gets the probability of the label
          if probability > 0:
              entropy += -probability * numpy.log2(probability)
      return entropy
      
    def iGain(self, pEntropy, weigh, entropy):
      return pEntropy - numpy.sum(numpy.array(weigh) * numpy.array(entropy)) # calculates the information gain



In [ ]:

class DTreeClassifier:
    def __init__(self, data, label):
        self.data = data
        self.label = label
        self.tree = tree()

    def predict(self, features, depth):
        self.tree.setDepth(depth)
        return self.tree.createTree(self.data, features)

    def predictAll(self, features_list, depth):
        predictions = []
        for features in features_list:
            predictions.append(self.predict(features, depth))
        return predictions

    def accuracy(self, labels, predictions):
        correct = 0
        for i in range(len(labels)):
            if labels[i] == predictions[i]:
                correct += 1
        return correct / len(labels)